In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import keras_tuner
import keras

In [2]:
csv_file_path = './airbnb_dataset/milestone3.csv'
df = pd.read_csv(csv_file_path)
pd.set_option('display.max_columns', None)
df.head()

,log_price,accommodates,bathrooms,host_response_rate,bedrooms,beds,cleaning_fee_True,host_has_profile_pic_t,host_identity_verified_t,instant_bookable_t,property_type_encoded,room_type_encoded,bed_type_encoded,cancellation_policy_encoded,city_encoded,des_sentiment_analysis,name_sentiment_analysis
0,5.010635,-0.072621,-0.404046,0.296013,-0.312048,-0.566461,True,True,True,False,-0.245313,0.876836,0.168067,1.021281,-0.434017,0.216242,-0.435248
1,5.129899,1.783653,-0.404046,0.296013,2.034955,1.027816,True,True,False,True,-0.245338,0.876830,0.168044,1.021245,-0.434042,0.837002,-1.788095
2,4.976734,0.855516,-0.404046,0.296013,-0.312048,1.027816,True,True,True,True,-0.245305,0.876838,0.168074,-0.593873,-0.434009,0.808508,-0.133695
3,6.620073,0.391448,-0.404046,0.296013,0.861454,0.230678,True,True,True,False,0.157557,0.876747,0.167757,-1.013554,2.675098,0.213627,-0.926212
4,4.744932,-0.536689,-0.404046,0.296013,-1.485549,-0.566461,True,True,True,True,-0.245255,0.876851,0.168119,-0.593754,1.412368,-0.101872,-0.095776


In [27]:
features = [col for col in df.columns if col != 'log_price' and df[col].dtype in ['int64', 'float64']]
features.remove('name_sentiment_analysis')
X = df[features]
y = df['log_price']

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [29]:
X_train[:5]

,accommodates,bathrooms,host_response_rate,bedrooms,beds,property_type_encoded,room_type_encoded,bed_type_encoded,cancellation_policy_encoded,city_encoded,des_sentiment_analysis
41089,-0.072621,-0.404046,0.296013,-1.485549,0.230678,-0.245263,0.876849,0.168112,-0.593772,-0.425992,0.388023
16409,-0.536689,-0.404046,0.296013,-0.312048,-0.566461,-2.389515,-1.035218,0.168127,-0.593732,-0.425968,0.128472
8929,-0.536689,-0.404046,0.296013,-0.312048,-0.566461,0.158913,-1.035203,0.168166,1.021435,1.412665,-1.062835
71528,0.855516,-0.404046,-0.120489,-0.312048,1.027816,-0.245298,0.876840,0.168081,-0.593855,-0.426042,-0.006515
51597,0.391448,-0.404046,0.296013,-0.312048,0.230678,2.388691,-2.097751,-6.355422,-0.593629,-0.425906,-1.203702


### Train with NN

In [30]:
# Uncomment below if tensorflow is not installed already
#%pip install tensorflow
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [31]:
# Build Neural Network
nn_model = Sequential([
    Dense(32, activation='relu',input_dim=X_train.shape[1]),
    Dense(16, activation='relu'),
    Dense(8, activation='relu'),
    Dense(1)
])

nn_model.compile(optimizer='adam', loss='mean_squared_error')

In [32]:
# Train the model
history = nn_model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/20


1849/1849 [==============================] - 1s 621us/step - loss: 1.2778 - val_loss: 0.2202
Epoch 2/20
1849/1849 [==============================] - 1s 567us/step - loss: 0.2112 - val_loss: 0.2281
Epoch 3/20
1849/1849 [==============================] - 1s 552us/step - loss: 0.2036 - val_loss: 0.2024
Epoch 4/20
1849/1849 [==============================] - 1s 553us/step - loss: 0.1999 - val_loss: 0.2043
Epoch 5/20
1849/1849 [==============================] - 1s 566us/step - loss: 0.1986 - val_loss: 0.1985
Epoch 6/20
1849/1849 [==============================] - 1s 555us/step - loss: 0.1973 - val_loss: 0.2154
Epoch 7/20
1849/1849 [==============================] - 1s 549us/step - loss: 0.1960 - val_loss: 0.1976
Epoch 8/20
1849/1849 [==============================] - 1s 568us/step - loss: 0.1953 - val_loss: 0.1996
Epoch 9/20
1849/1849 [==============================] - 1s 557us/step - loss: 0.1945 - val_loss: 0.2045
Epoch 10/20
1849/1849 [==============================] - 1s 566us/step - lo

In [33]:
# Evaluate the model
mse = nn_model.evaluate(X_test, y_test)
print(f"Mean Squared Error on Test Set: {mse}")

463/463 [==============================] - 0s 315us/step - loss: 0.1961
Mean Squared Error on Test Set: 0.19605639576911926


### Hyper Tuning

In [34]:
metrics = [keras.metrics.MeanSquaredError(name="mean_squared_error", dtype=None)]
def build_hp_model(hp):
    model = Sequential()
    for i in range(hp.Int('num_layers', 2, 4)):
        model.add(Dense(units=hp.Int('units_' + str(i), min_value=32, max_value=128, step=32),
                        activation='relu'))
    model.add(Dense(1))
    learning_rate = hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='mean_squared_error',
                  metrics=['mean_squared_error'])
    return model

In [35]:
early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0.001,
    patience=5,
    mode='auto',
    baseline=None,
    restore_best_weights=False,
    start_from_epoch=0
)

model_checkpoint = keras.callbacks.ModelCheckpoint(
    filepath='checkpoints',
    monitor='val_loss',
    save_best_only=True,
    save_weights_only= False,
    mode='auto'
)

In [36]:
tuner = keras_tuner.RandomSearch(
    hypermodel=build_hp_model,
    objective='val_loss',
    max_trials=5,
    seed=10,
    executions_per_trial=3,
    overwrite=True,
    tune_new_entries=True,
    allow_new_entries=True,
    max_consecutive_failed_trials=1
)

In [37]:
# Start searching
tuner_search = tuner.search(X_train, y_train, epochs=15, callbacks=[early_stopping, model_checkpoint], validation_split=0.2)

Trial 5 Complete [00h 00m 49s]
val_loss: 0.18971705436706543

Best val_loss So Far: 0.18812736372152963
Total elapsed time: 00h 04m 19s


In [ ]:
# View summary of searching
# tuner.results_summary()

In [38]:
# View best hyperparameters
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]
print("Best hyperparameters found were: ", best_hyperparameters.values)

Best hyperparameters found were:  {'num_layers': 4, 'units_0': 64, 'units_1': 96, 'learning_rate': 0.0019815016575264635, 'units_2': 32, 'units_3': 96}


In [39]:
best_model = tuner.get_best_models(num_models=1)[0]
mse = best_model.evaluate(X_test, y_test)
print(f"Mean Squared Error on Test Set: {mse}")

463/463 [==============================] - 0s 348us/step - loss: 0.1918 - mean_squared_error: 0.1918
Mean Squared Error on Test Set: [0.19181105494499207, 0.19181105494499207]


In [ ]:
# View best model
# best_model = tuner.get_best_models()[0]
# best_model.summary()


In [ ]:
# Evaluate the best model
# loss, acc = best_model.evaluate(X_test, y_test)
# print('ACC: ', acc)
# print('LOSS: ', loss)